In [91]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)

In [92]:
df = pd.read_csv('./dataset/brca_metabric_clinical_data.tsv', sep='\t')
df.to_csv('./dataset/brca.csv')

In [93]:
df.drop(columns=['Study ID', 'Patient ID', 'Sample ID'], inplace=True)

In [94]:
n_nan_columns = df.isnull().sum()
print(n_nan_columns.head(50))

Age at Diagnosis                   11
Type of Breast Surgery            554
Cancer Type                         0
Cancer Type Detailed                0
Cellularity                       592
Chemotherapy                      529
Pam50 + Claudin-low subtype       529
Cohort                             11
ER status measured by IHC          83
ER Status                          40
Neoplasm Histologic Grade         121
HER2 status measured by SNP6      529
HER2 Status                       529
Tumor Other Histologic Subtype    135
Hormone Therapy                   529
Inferred Menopausal State         529
Integrative Cluster               529
Primary Tumor Laterality          639
Lymph nodes examined positive     266
Mutation Count                    151
Nottingham prognostic index       222
Oncotree Code                       0
Overall Survival (Months)         528
Overall Survival Status           528
PR Status                         529
Radio Therapy                     529
Relapse Free

In [95]:
df.dropna(inplace=True)

In [96]:
df

,Age at Diagnosis,Type of Breast Surgery,Cancer Type,Cancer Type Detailed,Cellularity,Chemotherapy,Pam50 + Claudin-low subtype,Cohort,ER status measured by IHC,ER Status,Neoplasm Histologic Grade,HER2 status measured by SNP6,HER2 Status,Tumor Other Histologic Subtype,Hormone Therapy,Inferred Menopausal State,Integrative Cluster,Primary Tumor Laterality,Lymph nodes examined positive,Mutation Count,Nottingham prognostic index,Oncotree Code,Overall Survival (Months),Overall Survival Status,PR Status,Radio Therapy,Relapse Free Status (Months),Relapse Free Status,Number of Samples Per Patient,Sample Type,Sex,3-Gene classifier subtype,TMB (nonsynonymous),Tumor Size,Tumor Stage,Patient's Vital Status
1,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,NO,LumA,1.0,Positve,Positive,3.0,NEUTRAL,Negative,Ductal/NST,YES,Pre,4ER+,Right,0.0,2.0,4.020,IDC,84.633333,0:LIVING,Positive,YES,83.52,0:Not Recurred,1,Primary,Female,ER+/HER2- High Prolif,2.615035,10.0,1.0,Living
4,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,YES,LumB,1.0,Positve,Positive,3.0,NEUTRAL,Negative,Mixed,YES,Post,9,Right,8.0,2.0,6.080,MDLC,41.366667,1:DECEASED,Positive,YES,18.55,1:Recurred,1,Primary,Female,ER+/HER2- High Prolif,2.615035,40.0,2.0,Died of Disease
5,78.77,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,NO,LumB,1.0,Positve,Positive,3.0,NEUTRAL,Negative,Ductal/NST,YES,Post,7,Left,0.0,4.0,4.062,IDC,7.800000,1:DECEASED,Positive,YES,2.89,1:Recurred,1,Primary,Female,ER+/HER2- High Prolif,5.230071,31.0,4.0,Died of Disease
10,86.41,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,NO,LumB,1.0,Positve,Positive,3.0,GAIN,Negative,Ductal/NST,YES,Post,9,Right,1.0,4.0,5.032,IDC,36.566667,1:DECEASED,Negative,YES,36.09,0:Not Recurred,1,Primary,Female,ER+/HER2- High Prolif,5.230071,16.0,2.0,Died of Other Causes
11,84.22,MASTECTOMY,Breast Cancer,Breast Invasive Lobular Carcinoma,High,NO,Her2,1.0,Negative,Positive,2.0,LOSS,Negative,Lobular,NO,Post,3,Left,0.0,5.0,3.056,ILC,36.266667,1:DECEASED,Negative,NO,35.79,1:Recurred,1,Primary,Female,ER+/HER2- High Prolif,6.537589,28.0,2.0,Died of Disease
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,71.22,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,NO,LumA,5.0,Positve,Positive,2.0,NEUTRAL,Negative,Mixed,YES,Post,3,Left,4.0,11.0,5.060,MDLC,85.000000,1:DECEASED,Positive,NO,83.88,0:Not Recurred,1,Primary,Female,ER+/HER2- Low Prolif,14.382695,30.0,2.0,Died of Other Causes
1698,70.65,BREAST CONSERVING,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,NO,LumB,5.0,Positve,Positive,1.0,NEUTRAL,Negative,Mixed,NO,Post,8,Left,0.0,9.0,2.040,MDLC,201.166667,0:LIVING,Positive,YES,198.52,0:Not Recurred,1,Primary,Female,ER+/HER2- Low Prolif,11.767659,20.0,1.0,Living
1700,75.62,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,NO,Basal,5.0,Negative,Negative,3.0,GAIN,Negative,Ductal/NST,NO,Post,10,Right,0.0,4.0,4.040,IDC,105.200000,1:DECEASED,Negative,NO,103.82,0:Not Recurred,1,Primary,Female,ER-/HER2-,5.230071,20.0,1.0,Died of Other Causes
1702,52.84,BREAST CONSERVING,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,YES,Normal,5.0,Positve,Positive,2.0,NEUTRAL,Negative,Mixed,NO,Post,8,Right,6.0,5.0,5.040,MDLC,200.333333,0:LIVING,Positive,YES,197.70,0:Not Recurred,1,Primary,Female,ER+/HER2- High Prolif,6.537589,20.0,2.0,Living


In [97]:
X = df.iloc[:, :-2]
y = df.iloc[:, -2]

In [98]:
X = pd.get_dummies(X)

In [99]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

accuracy


0.8904109589041096

In [101]:
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)

accuracy_svm

0.7442922374429224

In [103]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
accuracies = []

for kernel in kernels:
    svm_model = SVC(kernel=kernel)
    svm_model.fit(X_train, y_train)
    y_pred_svm = svm_model.predict(X_test)
    accuracy_svm = accuracy_score(y_test, y_pred_svm)
    accuracies.append(accuracy_svm)
    print(kernel, accuracy_svm)

best_kernel = kernels[accuracies.index(max(accuracies))]
best_accuracy = max(accuracies)

best_kernel, best_accuracy


linear 0.7945205479452054
poly 0.6757990867579908
rbf 0.7442922374429224
sigmoid 0.7945205479452054


('linear', 0.7945205479452054)

In [90]:
correlation_matrix = X.corr()
correlation_matrix
# highly_correlated_features = correlation_matrix[abs(correlation_matrix) > 0.8].stack().index.tolist()


,Age at Diagnosis,Cohort,Neoplasm Histologic Grade,Lymph nodes examined positive,Mutation Count,Nottingham prognostic index,Overall Survival (Months),Relapse Free Status (Months),Number of Samples Per Patient,TMB (nonsynonymous),Tumor Size,Type of Breast Surgery_BREAST CONSERVING,Type of Breast Surgery_MASTECTOMY,Cancer Type_Breast Cancer,Cancer Type Detailed_Breast,Cancer Type Detailed_Breast Invasive Ductal Carcinoma,Cancer Type Detailed_Breast Invasive Lobular Carcinoma,Cancer Type Detailed_Breast Invasive Mixed Mucinous Carcinoma,Cancer Type Detailed_Breast Mixed Ductal and Lobular Carcinoma,Cellularity_High,Cellularity_Low,Cellularity_Moderate,Chemotherapy_NO,Chemotherapy_YES,Pam50 + Claudin-low subtype_Basal,Pam50 + Claudin-low subtype_Her2,Pam50 + Claudin-low subtype_LumA,Pam50 + Claudin-low subtype_LumB,Pam50 + Claudin-low subtype_NC,Pam50 + Claudin-low subtype_Normal,Pam50 + Claudin-low subtype_claudin-low,ER status measured by IHC_Negative,ER status measured by IHC_Positve,ER Status_Negative,ER Status_Positive,HER2 status measured by SNP6_GAIN,HER2 status measured by SNP6_LOSS,HER2 status measured by SNP6_NEUTRAL,HER2 status measured by SNP6_UNDEF,HER2 Status_Negative,HER2 Status_Positive,Tumor Other Histologic Subtype_Ductal/NST,Tumor Other Histologic Subtype_Lobular,Tumor Other Histologic Subtype_Medullary,Tumor Other Histologic Subtype_Mixed,Tumor Other Histologic Subtype_Mucinous,Tumor Other Histologic Subtype_Other,Tumor Other Histologic Subtype_Tubular/ cribriform,Hormone Therapy_NO,Hormone Therapy_YES,Inferred Menopausal State_Post,Inferred Menopausal State_Pre,Integrative Cluster_1,Integrative Cluster_10,Integrative Cluster_2,Integrative Cluster_3,Integrative Cluster_4ER+,Integrative Cluster_4ER-,Integrative Cluster_5,Integrative Cluster_6,Integrative Cluster_7,Integrative Cluster_8,Integrative Cluster_9,Primary Tumor Laterality_Left,Primary Tumor Laterality_Right,Oncotree Code_BREAST,Oncotree Code_IDC,Oncotree Code_ILC,Oncotree Code_IMMC,Oncotree Code_MDLC,Overall Survival Status_0:LIVING,Overall Survival Status_1:DECEASED,PR Status_Negative,PR Status_Positive,Radio Therapy_NO,Radio Therapy_YES,Relapse Free Status_0:Not Recurred,Relapse Free Status_1:Recurred,Sample Type_Primary,Sex_Female,3-Gene classifier subtype_ER+/HER2- High Prolif,3-Gene classifier subtype_ER+/HER2- Low Prolif,3-Gene classifier subtype_ER-/HER2-,3-Gene classifier subtype_HER2+
Age at Diagnosis,1.000000,0.031611,-0.096408,0.025244,0.032214,-0.026063,-0.163522,-0.098289,NaN,0.031076,0.064657,-0.106074,0.106074,NaN,-0.018473,-0.050553,0.045938,0.001387,0.030238,0.001852,-0.002958,0.000026,0.413432,-0.413432,-0.131482,-0.083122,0.065528,0.176390,0.022667,-0.109099,-0.070598,-0.257898,0.257898,-0.218024,0.218024,-0.093138,0.032310,0.068773,0.019469,0.133792,-0.133792,-0.037677,0.045938,-0.011374,0.030238,0.001387,-0.018473,-0.029287,-0.284142,0.284142,0.746013,-0.746013,-0.062088,-0.166168,0.077695,0.047245,0.016625,-0.029235,-0.119722,0.026465,0.099650,0.063043,0.054816,0.038351,-0.038351,-0.018473,-0.050553,0.045938,0.001387,0.030238,-0.300358,0.300358,-0.052677,0.052677,0.079122,-0.079122,0.061335,-0.061335,NaN,NaN,0.131150,0.057212,-0.130077,-0.132104
Cohort,0.031611,1.000000,-0.011275,-0.137843,0.304067,-0.113454,0.193670,0.141868,NaN,0.304784,-0.059629,0.058305,-0.058305,NaN,0.026939,-0.059870,-0.007468,-0.016526,0.079494,0.027417,-0.031333,-0.007769,0.233979,-0.233979,0.064224,0.025519,0.000452,-0.006717,0.012529,-0.003997,-0.078249,0.059014,-0.059014,0.029173,-0.029173,0.013440,-0.036341,0.002125,0.034957,-0.015861,0.015861,-0.068352,-0.007468,0.045367,0.079494,-0.016526,0.026939,-0.002558,0.243939,-0.243939,0.014575,-0.014575,0.007516,0.002592,-0.024595,0.029715,-0.018073,0.000879,0.017372,-0.029885,0.018236,0.012805,-0.044366,0.036630,-0.036630,0.026939,-0.059870,-0.007468,-0.016526,0.079494,-0.194617,0.194617,-0.001887,0.001887,0.164449,-0.164449,-0.055598,0.055598,NaN,NaN,-0.032474,-0.012496,0.038784,0.022289
Neoplasm Histologic

In [104]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
X_pca = pca.fit_transform(X)

X_pca


array([[-1.23665044,  0.2202125 , -3.09361505,  1.02745183,  1.77974988],
       [-1.59454971, -1.76233055,  4.08218269, -3.64502699,  2.05577789],
       [-1.17956251, -3.58123623, -0.62675025, -2.4791757 ,  0.1068351 ],
       ...,
       [ 4.42243899,  2.02382307, -0.11360488, -1.54316362, -2.42986336],
       [-2.51213788,  3.17292885,  2.08430487,  0.60336875,  0.65321312],
       [-4.10198652,  3.1148563 , -0.62134321,  3.24305329,  6.25205119]])

In [105]:
explained_variance = pca.explained_variance_ratio_
variance_covered = sum(explained_variance[:5]) * 100
variance_covered

35.529029449549874

In [86]:
le = LabelEncoder()
y_train_transformed = le.fit_transform(y_train)
y_test_transformed = le.fit_transform(y_test)

print(y_train_transformed)
print(y_train)


xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train_transformed)

y_pred_xgb = xgb_model.predict(y_test_transformed)
accuracy_xgb = accuracy_score(y_test_transformed, y_pred_xgb)

accuracy_xgb

[1 1 0 0 1 0 1 2 0 2 0 0 1 0 0 0 1 1 0 1 1 1 1 2 1 0 1 1 1 0 0 1 1 1 0 1 1
 1 1 0 1 1 0 0 0 0 1 1 1 1 1 0 0 3 0 1 1 1 1 1 0 1 1 0 0 1 1 1 2 1 0 1 1 0
 0 1 1 2 1 2 1 0 1 0 0 2 1 1 0 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 0 0 1 0
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 0 0 0 3 2 1 1 1 1 1 1 0 0 0 0
 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 2 1 0 1 0 2 0 1 1 0 1 0 0 0 1 1 0 0 1 2 1 1
 0 2 1 1 2 1 1 1 1 1 1 0 1 1 1 1 1 2 1 1 1 0 1 1 0 0 1 2 1 1 0 1 1 2 0 1 1
 1 1 1 0 1 0 2 0 1 1 1 0 0 0 1 2 0 0 0 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1
 0 0 1 0 1 2 1 2 1 0 0 1 1 0 1 1 0 1 1 0 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 1
 1 1 1 1 1 1 2 1 1 1 0 1 0 1 1 2 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 2 1 1 1 1
 0 1 0 1 1 0 1 1 0 1 1 1 2 0 2 1 0 2 1 1 0 0 1 1 0 0 1 1 1 2 1 0 1 0 0 0 1
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0 1 1 1 0 0 1 1 1 0 0 0 1 0 1
 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 2 0 1 1 1 1 0 1 0 1 1
 1 0 0 1 1 1 1 1 1 0 1 2 0 1 2 1 0 0 1 1 0 0 0 2 1 1 1 1 1 0 0 0 1 0 1 0 1
 1 2 1 0 0 1 1 1 1 0 0 1 

XGBoostError: [11:54:40] /Users/runner/work/xgboost/xgboost/src/predictor/cpu_predictor.cc:719: Check failed: m->NumColumns() == model.learner_model_param->num_feature (1 vs. 84) : Number of columns in data must equal to trained model.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000145edb355 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x0000000146123497 void xgboost::predictor::CPUPredictor::DispatchedInplacePredict<xgboost::data::ArrayAdapter, 64ul>(std::__1::any const&, std::__1::shared_ptr<xgboost::DMatrix>, xgboost::gbm::GBTreeModel const&, float, xgboost::PredictionCacheEntry*, unsigned int, unsigned int) const + 407
  [bt] (2) 3   libxgboost.dylib                    0x0000000146118147 xgboost::predictor::CPUPredictor::InplacePredict(std::__1::shared_ptr<xgboost::DMatrix>, xgboost::gbm::GBTreeModel const&, float, xgboost::PredictionCacheEntry*, unsigned int, unsigned int) const + 1463
  [bt] (3) 4   l